### データの読み込み
表形式のデータを扱うのに長けた，pandasでデータを読み込む。
pandasにはread_csv()やread_excel()関数があり，一般的な表形式データをそのまま読み込むことができる。

In [20]:
import pandas as pd
df = pd.read_csv("./CarPrice_Assignment.csv")

Jupyter Notebook形式でスクリプトを書いている場合，セルの最終行に変数名だけを記載した場合，その内容が表示される。
その他の場所で表示がしたい場合はprint()関数の使用が必要

In [21]:
df

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


## データの整形
ダウンロードしたままの形出は，ニューラルネットワークへ直接入力を行うことができない。ここでは入力できる形へデータを整形する。

## 予測に意味をなさないデータの除外
読み込んだデータには，車の名称など，直接的に価格に結びつきにくいデータが含まれている。そういったデータを除外する。

※言語モデルを用いて「ポルシェだから高そう」や「高級そうな車種名だ」といった情報を加えたい場合，話は別
※単純に数値かする上でも，車種名を省いてメーカー名のみ抽出すれば情報として有意義かもしれない

In [ ]:
# 学習から除外する列を指定
drop_columns = ["car_ID", "CarName"]

# dfをコピー
df_buf = df.drop(drop_columns, axis=1)

### 文字列で記録されたデータの数値化
読みこんだデータには，文字列が多分に含まれている。先週扱った通り，ニューラルネットワークは「数値（数は問わない）」を入力して「数値（数は問わない）」を出力する。従って，例えば燃料タイプの"gas"や"diesel"は，何かしらの数値へ変換する必要がある。ここでは，アルファベット順にソートした名称順に0,1,2...と数値化を行う。

In [ ]:
# 列に対してループ
for column in df_buf.columns:
    
    # 列名称とデータ型を表示
    print(f"-----------------------------------------")
    print(f"col name: {column}, dtype: {df_buf[column].dtype}")

    # ユニークな値のリストを表示
    print(f"unique values:{df_buf[column].unique()}")
    
    # object型の列に対しては，ユニークな値を0始まりの整数に置換
    if df_buf[column].dtype == "object":
        df_buf[column] = pd.Categorical(df[column])
        df_buf[column] = df_buf[column].cat.codes
        print(f"convert:{df_buf[column].unique()}")


-----------------------------------------
col name: symboling, dtype: int64
unique values:[ 3  1  2  0 -1 -2]
-----------------------------------------
col name: fueltype, dtype: object
unique values:['gas' 'diesel']
convert:[1 0]
-----------------------------------------
col name: aspiration, dtype: object
unique values:['std' 'turbo']
convert:[0 1]
-----------------------------------------
col name: doornumber, dtype: object
unique values:['two' 'four']
convert:[1 0]
-----------------------------------------
col name: carbody, dtype: object
unique values:['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
convert:[0 2 3 4 1]
-----------------------------------------
col name: drivewheel, dtype: object
unique values:['rwd' 'fwd' '4wd']
convert:[2 1 0]
-----------------------------------------
col name: enginelocation, dtype: object
unique values:['front' 'rear']
convert:[0 1]
-----------------------------------------
col name: wheelbase, dtype: float64
unique values:[ 88.6  94.5  99

## データの数値レンジの正規化
今回のデータにおいては，燃料タイプは0-1で，重量は2548-3062の範囲で分布している。
これらのデータの最大値と最小値の差は，1と514で，このままモデルに入力を行った場合，このバランスを取るために訓練のリソース消費される。
意図的に何かのデータを重視したい場合を除いては，これらの値は同じスケールに統一したほうがよい。
そういった処理としてMin-Max normalization という，最小値を0，最大値を1で統一する正規化手法が存在する。

In [28]:
for column in df_buf.columns:
    print(f"col name: {column}, min: {df_buf[column].min()}, max: {df_buf[column].max()}")
    df_buf[column] = (df_buf[column] - df_buf[column].min()) / (df_buf[column].max() - df_buf[column].min())

col name: symboling, min: 0.0, max: 1.0
col name: fueltype, min: 0.0, max: 1.0
col name: aspiration, min: 0.0, max: 1.0
col name: doornumber, min: 0.0, max: 1.0
col name: carbody, min: 0.0, max: 1.0
col name: drivewheel, min: 0.0, max: 1.0
col name: enginelocation, min: 0.0, max: 1.0
col name: wheelbase, min: 0.0, max: 1.0
col name: carlength, min: 0.0, max: 1.0
col name: carwidth, min: 0.0, max: 1.0
col name: carheight, min: 0.0, max: 1.0
col name: curbweight, min: 0.0, max: 1.0
col name: enginetype, min: 0.0, max: 1.0
col name: cylindernumber, min: 0.0, max: 1.0
col name: enginesize, min: 0.0, max: 1.0
col name: fuelsystem, min: 0.0, max: 1.0
col name: boreratio, min: 0.0, max: 1.0
col name: stroke, min: 0.0, max: 1.0
col name: compressionratio, min: 0.0, max: 1.0
col name: horsepower, min: 0.0, max: 1.0
col name: peakrpm, min: 0.0, max: 1.0
col name: citympg, min: 0.0, max: 1.0
col name: highwaympg, min: 0.0, max: 1.0
col name: price, min: 0.0, max: 1.0
